# Update distortion CDP version 06.04.00 to version 7B.05.00

In [1]:
import datetime
import numpy as np
from astropy.io import fits
import numpy.lib.recfunctions as rec

import warnings
warnings.simplefilter('ignore')

In [2]:
# Define paths to data
workDir   = '/Users/ioannisa/Desktop/python/miri_devel/'
cdpDir    = workDir+'cdp_data/'
outDir    = cdpDir+"CDP7/"

In [3]:
def convert_2x2_to_3x3(label,a,b,c,d):
    return [label,a,b,0,c,d,0,0,0,0]

In [14]:
# create all new distortion cdps
for miri_setup in ['IFUSHORT_12SHORT','IFUSHORT_12MEDIUM','IFUSHORT_12LONG','IFULONG_34SHORT','IFULONG_34MEDIUM','IFULONG_34LONG']:
    # open slice map pseudo-cdp
    hdulist_slicemap = fits.open(cdpDir+'CDP7/MIRI_FM_MIR{}_SLICEMAP_7B.00.00.fits'.format(miri_setup))
    
    # initialize cube placeholder
    new_sliceMaps = np.zeros((9,1024,1032)) # Make a 1032x1024 array with a total of nine layers (different transmissions)
    
    # create slice map cube
    layer = -1
    for transmission in ['10','20','30','40','50','60','70','80','90']:
        layer +=1
        new_sliceMaps[layer,:,:] = hdulist_slicemap[layer+1].data
        
    # load old distortion cdp fits file
    old_distortion_cdp = fits.open(cdpDir+'CDP7/MIRI_FM_MIR{}_DISTORTION_06.04.00.fits'.format(miri_setup))
    
    # change headers
    old_distortion_cdp[0].header['DATE']     = datetime.datetime.utcnow().isoformat()
    old_distortion_cdp[0].header['FILENAME'] = 'MIRI_FM_MIR{}_DISTORTION_7B.05.00.fits'.format(miri_setup)
    old_distortion_cdp[0].header['VERSION']  = '7B.05.00'
    old_distortion_cdp[0].header.add_history("- created new slice maps based on slice transmission -> DOCUMENT:")
    old_distortion_cdp[0].header.add_history("- MIRI-TN-00005-KUL, Ioannis Argyriou and Bart Vandenbussche")
    old_distortion_cdp[0].header['ORIGFILE'] = 'MIRI_FM_MIR{}_DISTORTION_06.04.00.fits'.format(miri_setup)
    old_distortion_cdp[0].header['USEAFTER'] = "2000-01-01T00:00:00"
    old_distortion_cdp[0].header.add_history("- increased the power in the albe_to_XANYAN matrices from i,j = 0,1 to")
    old_distortion_cdp[0].header.add_history("- i,j = 0,1,2")
    old_distortion_cdp[0].header.add_history("- increased the power in the XANYAN_to_albe matrices from i,j = 0,1 to")
    old_distortion_cdp[0].header.add_history("- i,j = 0,1,2")

    # extend definition of transform matrices from alpha-beta to XAN/YAN and vice-versa
    albe_to_XANYAN = old_distortion_cdp['albe_to_XANYAN'].data.copy()
    XANYAN_to_albe = old_distortion_cdp['XANYAN_to_albe'].data.copy()

    albe_to_XANYAN_new = np.rec.array([convert_2x2_to_3x3(*list(albe_to_XANYAN[0])),
                  convert_2x2_to_3x3(*list(albe_to_XANYAN[1])),
                  convert_2x2_to_3x3(*list(albe_to_XANYAN[2])),
                  convert_2x2_to_3x3(*list(albe_to_XANYAN[3]))],dtype=[('LABEL', 'S16'), ('COEFF_0_0', 'float64'), ('COEFF_0_1', 'float64'), ('COEFF_0_2', 'float64'), ('COEFF_1_0', 'float64'), ('COEFF_1_1', 'float64'), ('COEFF_1_2', 'float64'), ('COEFF_2_0', 'float64'), ('COEFF_2_1', 'float64'), ('COEFF_2_2', 'float64')])

    XANYAN_to_albe_new = np.rec.array([convert_2x2_to_3x3(*list(XANYAN_to_albe[0])),
                  convert_2x2_to_3x3(*list(XANYAN_to_albe[1])),
                  convert_2x2_to_3x3(*list(XANYAN_to_albe[2])),
                  convert_2x2_to_3x3(*list(XANYAN_to_albe[3]))],dtype=[('LABEL', 'S16'), ('COEFF_0_0', 'float64'), ('COEFF_0_1', 'float64'), ('COEFF_0_2', 'float64'), ('COEFF_1_0', 'float64'), ('COEFF_1_1', 'float64'), ('COEFF_1_2', 'float64'), ('COEFF_2_0', 'float64'), ('COEFF_2_1', 'float64'), ('COEFF_2_2', 'float64')])

    # update corresponding fits extensions
    old_distortion_cdp['albe_to_XANYAN'].data = albe_to_XANYAN_new
    old_distortion_cdp['XANYAN_to_albe'].data = XANYAN_to_albe_new
    old_distortion_cdp['Slice_number'].data   = new_sliceMaps.copy()
    
    for level in range(1,10):
        old_distortion_cdp['Slice_number'].header['LEVEL_{}'.format(str(level))] = '{}% transmission slice map'.format(str(int(level*10.)))

    # save output
    old_distortion_cdp.writeto(outDir + "MIRI_FM_MIR{}_DISTORTION_7B.05.00.fits".format(miri_setup),overwrite=True)